In [1]:
from emo_classifier import EmoClassifier
from gaze_estimation import yolo_face, mobile_gaze, pre_process

2025-12-03 00:06:03.822098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 00:06:04.795358: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
import cv2

In [3]:
device = 'cuda'
face_detector = yolo_face(device)
gaze_detector = mobile_gaze(device)
emo_classifier = EmoClassifier(
        detector_backend='mtcnn',
        enforce_detection=False
    )

cap = cv2.VideoCapture("vidos.webm")
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter("output.avi", fourcc, 30.0, (frame_width, frame_height))
a = 0
while 1:
    ret, unprocessed = cap.read()
    if not ret:
        break
    img = pre_process(unprocessed)

    bbox_list = face_detector.forward(img)
    face_batch = face_detector.make_face_batch(unprocessed, bbox_list, pre_process)
    if face_batch.shape[0]:
        pitch, yaw = gaze_detector.forward(face_batch.to(device))
        gaze_detector.draw_result(unprocessed, pitch, yaw, bbox_list)

        emo_classifier = EmoClassifier(
            detector_backend='mtcnn',
            enforce_detection=False
        )
        emotions = emo_classifier.predict(face_batch)
        unprocessed = emo_classifier.draw_result(unprocessed, bbox_list, emotions)

    out.write(unprocessed)
    print(a, end = '\r')
    a += 1
cap.release()
out.release()

2025-12-03 00:06:08.841876: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1764695168.842887  117814 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9125 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-12-03 00:06:09.853539: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600


KeyboardInterrupt: 